## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-09-01-13-50 +0000,nyt,Suspect and Officer Are Dead After Shooting Ou...,https://www.nytimes.com/2025/08/08/us/emory-un...
1,2025-08-09-01-09-00 +0000,wsj,Fast-Moving Wildfires Threaten L.A. County,https://www.wsj.com/us-news/california-canyon-...
2,2025-08-09-01-07-00 +0000,wsj,"Stock Market News, Aug. 8, 2025: Gold Pares Ga...",https://www.wsj.com/livecoverage/stock-market-...
3,2025-08-09-01-02-37 +0000,latimes,Commentary: Disney's settlement with 'Mandalor...,https://www.latimes.com/entertainment-arts/sto...
4,2025-08-09-01-00-00 +0000,wsj,The bitter fight over redrawing U.S. House map...,https://www.wsj.com/politics/us-gerrymandering...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2246/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
52,trump,74
76,gaza,29
243,israel,19
75,plan,17
77,city,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
5,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...,184
170,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...,171
185,2025-08-08-14-53-06 +0000,nypost,Longtime foes Armenia and Azerbaijan ink histo...,https://nypost.com/2025/08/08/us-news/armenia-...,129
123,2025-08-08-19-22-00 +0000,wsj,President Trump’s team is reviewing new conten...,https://www.wsj.com/economy/central-banking/tr...,129
23,2025-08-08-23-56-11 +0000,bbc,Azerbaijan and Armenia sign peace deal at Whit...,https://www.bbc.com/news/articles/c39dzl1lzrgo...,127


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
5,184,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...
287,108,2025-08-08-03-37-58 +0000,wapo,Israel’s security cabinet approves plan for mi...,https://www.washingtonpost.com/world/2025/08/0...
170,69,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...
185,55,2025-08-08-14-53-06 +0000,nypost,Longtime foes Armenia and Azerbaijan ink histo...,https://nypost.com/2025/08/08/us-news/armenia-...
37,45,2025-08-08-23-04-25 +0000,cbc,Trump says he'll meet Putin in Alaska next Fri...,https://www.cbc.ca/news/world/trump-putin-meet...
160,39,2025-08-08-16-47-10 +0000,nypost,JD Vance goes fishing with top UK official Dav...,https://nypost.com/2025/08/08/us-news/jd-vance...
25,37,2025-08-08-23-49-00 +0000,wsj,President Trump removed former congressman Bil...,https://www.wsj.com/politics/policy/billy-long...
71,36,2025-08-08-21-31-22 +0000,nypost,Vicious NYC killing of 14-year-old Angel Mendo...,https://nypost.com/2025/08/08/us-news/vicious-...
177,34,2025-08-08-15-42-00 +0000,wsj,Trump’s Fed nominee would add a voice inside t...,https://www.wsj.com/economy/central-banking/st...
3,34,2025-08-09-01-02-37 +0000,latimes,Commentary: Disney's settlement with 'Mandalor...,https://www.latimes.com/entertainment-arts/sto...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
